In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeRegressor, plot_tree
import matplotlib.pyplot as plt

## Boosting

### Adaboost

In [ ]:
# データ準備
df = sns.load_dataset('titanic')
df.dropna(inplace=True)

# X, yを作成
X = df.drop(['survived', 'alive'], axis=1)
y = df['survived']

# ラベルエンコーディング
oe = 
oe.set_output(transform='pandas')
cat_cols = X.
X[cat_cols] = 

# hold-out
X_train, X_test, y_train, y_test = 
ada = 
y_pred_ada = 
print(f"adaboost AUC: {roc_auc_score(y_test, y_pred_ada[:, 1])}")

# 特徴量重要度
print(ada.feature_importances_)
plt.barh(X.columns, ada.feature_importances_)

### gradient boosting 回帰

### スクラッチで実装

In [ ]:
class MyGradientBoostingRegressor:
        
    # 引数はsklearnに合わせる
    def __init__():
    
    def fit(self, X, y):
        # 平均が最も損失を小さくする
        
    def predict(self, X):
        

In [ ]:
# データ準備
df = sns.load_dataset('mpg')
df.dropna(inplace=True)
X = df['horsepower'].values.reshape(-1, 1)
y = df['mpg'].values
# データの散布図描画
sns.scatterplot(x=df['horsepower'], y=df['mpg'])

In [ ]:
my_gbr = MyGradientBoostingRegressor()
my_gbr.fit(X, y)
my_gbr.predict(X)[:10]

###　sklearnのGradientBoostingRegressorを使う

In [ ]:
lr = 0.03
gbr = GradientBoostingRegressor(max_depth=, learning_rate=, random_state=0).fit(X, y)
# MyGradientBoostingRegressorの結果と同じであることを確認
gbr.predict(X)[:10]

In [ ]:
# 弱学習器の木構描画
model = gbr.estimators_[0][0]
_ = plot_tree(model)

In [ ]:
# 予測結果が弱学習器の組み合わせになっていることを確認する
results = []
# 一つのデータを選択し，そのデータを使って弱学習器の予測値を得て，最終的な予測値を計算し，.predictの結果と一致するか確認する
idx = 0
for t in 
    result = 
    results.append()

In [ ]:
F0 = np.mean(y)
Fb = F0 + np.cumsum(results)
Fb

In [ ]:
plt.plot(Fb)

### gradient boosting 分類

### スクラッチ実装

#### ヒント

In [ ]:
model = DecisionTreeRegressor(max_depth=1, random_state=0).fit(X, y)
# それぞれのデータがどの葉に落ちるかを取得
model.apply(X)

In [ ]:
# 決定木の葉(およびノード)の値
model.tree_.value

In [ ]:
# 上書き
model.tree_.value[1, 0, 0] = 3

In [ ]:
# id=1の値が3に上書きされている
plot_tree(model)

In [ ]:
# 予測結果も，id=1は全て3になっている
model.predict(X)[:30]

In [ ]:
class MyGradientBoostingClassifier:
    
    def __init__(self,
                 learning_rate=0.03,
                 n_estimators=100,
                 max_depth=1,
                 random_state=0
                 ):
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.estimators = []
        
    def fit(self, X, y):
        self.F0 = np.log(y.mean()/(1-y.mean()))
        F0 = np.full(len(y), F0)  # F0のvector化
        Fb = F0
        
        for _ in range(self.n_estimators):
            p = 1 / (1 + np.exp(-Fb))
            r = y - p
            estimator = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            estimator.fit(X, r)  # 残差rを目的変数として学習
            # model.tree_.valueの値を上書きすることによって,弱学習器の出力結果を上書き
            # 各葉のidを取得しそれぞれの葉に対して処理を行う
            X_leafs = estimator.apply(X)
            leaf_ids = np.unique(X_leafs)
            for leaf_id in leaf_ids:
                # 弱学習器の出力の値を計算
                fltr = X_leafs == leaf_id
                num = r[fltr].sum()
                # F0のときは全てのデータが同じpになるためscalerで良いが,
                # 1回目のiteration以降は各データXに別のpを割り当てるのでvectorで処理する必要がある
                # np.arrayの形に変換することで特定のデータをfilterして処理することが可能
                den = (p[fltr]*(1-p[fltr])).sum()
                estimator_pred_proba = num / den
                # 弱学習器の出力を上書き
                estimator.tree_.value[leaf_id, 0, 0] = estimator_pred_proba
                # 当該データのFbを更新
                Fb[fltr] += self.learning_rate * estimator_pred_proba

            self.estimators.append(estimator)

        
    def predict_proba(self, X):
        Fb = np.full(X.shape[0], self.F0)
        
        for estimator in self.estimators:
            Fb += self.learning_rate * estimator.predict(X)
        
        return 1 / (1 + np.exp(-Fb))

In [ ]:
my_gbc = MyGradientBoostingClassifier()
my_gbc.fit(X_train, y_train) # titanicデータセット
my_gbc.predict_proba(X_test)

### sklearnのGradientBoostingClassifierを使う

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.03, max_depth=1, random_state=0).fit(X_train, y_train)
# MyGradientBoostingClassifierの結果と等しいことを確認
gbc.predict_proba(X_test)[:, 1] == my_gbc.predict_proba(X_test)

In [ ]:
# 評価
print(f"grad boost AUC: {roc_auc_score(y_test, gbc.predict_proba(X_test)[:, 1])}")

In [ ]:
# 弱学習器の予測値からモデル全体の予測値が算出されることを確認する
results = []
for t in gbc.estimators_:
    result = 0.03 * t[0].predict([X_test.iloc[0]]) # idx=0のデータで確認
    results.append(result)
plt.plot(results)

In [ ]:
F0 = np.log(y_train.mean()/(1-y_train.mean()))
Fb = F0 + np.cumsum(results)[-1]
predict = 1 / (1 + np.exp(-Fb))
print(predict)

In [ ]:
# 弱学習器の数に対しての予測値の推移を描画
plt.plot(np.cumsum(results)+F0)